# find tracerRNA

example id:
8774__OBKKDPJE_00146__BritoIL_2016__M2.46.ST__bin.47;length1073;#sequences1 

In [4]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [5]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [6]:
seqid ="OBKKDPJE_00146"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  OBKKDPJE_00146
NODE_3_length_470079_cov_10.1699
BritoIL_2016
BritoIL_2016__M2.46.ST__bin.47
1073
[('.', '179452', '179677')]
----------------------------------------------------------------------


## Double cas check:

In [11]:
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
cas_dataset[cas_dataset["Genome Name"]==genomename]

,Seq ID,Seq Description,Seq,Contig,Genome Name,Study,Sample Name,SGB ID,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,uSGB,Level of estimated taxonomy,Estimated taxonomy
2652,OBKKDPJE_00146,OBKKDPJE_00146 CRISPR-associated endonuclease ...,MYTKEIIWGFDIGKGSLGEAVRVGNEFKHVQSLEIAPDFAETKTTA...,NODE_3_length_470079_cov_10.1699,BritoIL_2016__M2.46.ST__bin.47,BritoIL_2016,M2.46.ST,8774,174787 178008,"[('ID=CRISPR1', '179620', '3')]","[('.', '179452', '179677')]","[('ID=OBKKDPJE_00147', '178008', '178898')]","[('ID=OBKKDPJE_00148', '178941', '179336')]",Yes,Other,-


In [7]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")


########################################################################

# get CRISPR spacer and repeat sequence

s3_genomename, s3_dataset=filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset)
mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")

is_the_right_contig=False
for line in f.readlines():
    if line.startswith("Sequence \'"): #identifiy line of contigname
        if line.startswith("Sequence \'"+contigname): #check which contig we're on (if there is more than one)
            is_the_right_contig=True
        else:
            is_the_right_contig=False
    if is_the_right_contig:
        print(line)
        if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
        line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

            repeats.append(line.split("\t")[2])

            repeat_start_pos.append(line.split("\t")[0])

            if not line.split("\t")[3]=="\n":
                spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("+"*80)
print(cas_position)
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        temp_alt_cas9start=98503
        temp_alt_cas9stop=101700
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	OBKKDPJE_00146
SGB:	8774
Genome Name:	 BritoIL_2016__M2.46.ST__bin.47
Contig:	NODE_3_length_470079_cov_10.1699 
-----------------------------------------------------

Sequence 'NODE_3_length_470079_cov_10.1699' (470079 bp)



CRISPR 1   Range: 179452 - 179677

POSITION	REPEAT				SPACER

--------	----------------------------	--------------------------------------

179452		AATCTAAGAAAGAACACTTTGCTGTAAA	GTCAAAACTAATACGTTATAATCGGGGCGCGCCAGATG	[ 28, 38 ]

179518		AATCTAAGAAAGAACACTTTGCTGTAAA	GTCAAAACTTATGGCAAAATCTACAAAAATCGCAAACG	[ 28, 38 ]

179584		AATCTAAGAAAGAACACTTTGCTGTAAA	CTCAAAACTGGTTGCCATTTGGTTTGAAATTCCTCTGC	[ 28, 38 ]

179650		AATCTAAGGAAGGTTAGTTTGCTAAATA	

--------	----------------------------	--------------------------------------

Repeats: 4	Average Length: 28		Average Length: 38



Time to find repeats: 613 ms





Locus features and positions:  {'Cas9': [174787, 178008], 'Cas1': [178008, 178898], 'Cas2': [178941, 1

In [8]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['179452', '179518', '179584', '179650'] :
-----------------------------------------------------



## extract sequences NISBA

In [12]:
matched_rep_start=
matched_rep_stop=

In [39]:
# this is the sequence matching the repeat:
matched_rep=Seq("")

#rpts which match:
rpt2=Seq("")
rpt3=Seq("")

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

TTATACCATACCAAGAACTATTA


23

In [9]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("14"):
        print(line)
f.close()

141093  Rnamotif + TACGCGCTGCTTTTCCCGCGCGACGTCGGGGGATTTTTCGAATAC -7.60

141561  Rnamotif - ACCGACGGAGGCGAGTTTTGCAAGCTCGaTTTTTTTCAGCT -6.30

141733  Rnamotif - GTTGCGGACAAGCGGCAGGGGCAGAAGCTGCTTTTGGAGAAAG -6.30

143219  Rnamotif + CGGGCGGCGGGCGAGTGTTTTTTGTTGACGCTCGaTTTTTTAAGTCC -10.50

143277  Rnamotif - AAATCCGTTTGGCGCAGCTGAAAGCGCgTTTTAAATTAAG -6.80

143433  Rnamotif - TGACGGGAGCTTTTCCGGGGCGGAAGCCGGGAATTTTCGCGCTCT -9.70

144012  Rnamotif - ACTTCGTACTGCGCGGTCTTGCCCGCGaTTTTGTCGTTCT -8.40

145147  Rnamotif - TTCGTTAATTTTTTTGCGCCAGCGCAGAATTTCGTTCGCGG -6.50

145654  Rnamotif - GTAGGCCCTATGAGCAGCACGTTGCTCTTTTCGATTTCC -6.30

146008      Both - GCGGGTGTTTACCCTGATGTACTCTTGGTCGGGGTGTTTTCTTCCG -9.70

146563      Both - GGCAACAAAAAACCGCGCCGTTTCGGGCGCGGTtTTGCGTTTTGCGC -17.60

146566      Both + CAAAACGCAAAACCGCGCCCGAAACGGCGCGGTTTTTTGTTGCCCA -16.30

148823  Rnamotif - TACGATTTTCTTTTGCGGAAGCTTGCCCGCGAGcgTTTTTTCGAACG -8.60

149302  Rnamotif - GGCAAGCTCGGTGCCGTCGAGTCCGCGGTATTTTTTGGGAAC -5.90

149944  Rn

In [10]:
# this is the rho-independent trnascription terminator, going from 14719 or 14722 to bo (+1 perché conti il primo tipo), 

terminator_end=14652 # (sarebe 1463-1, for indexing reasons) sarebbe la fine xke prendiamo il rev comp remembra. che ha senso xke l'antirepeat è a 14729, che è dopo
 #oppure, ce n'è uno con TTTTTTT a 14653...
terminator=Seq("CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCaTCTTTTTTGTCA".upper())
print(len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

50


In [104]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb

# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)


pre_tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test repeat:  TTATACCATACCAAGAACTATTA 23 True
test terminator:  CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA 50 True
pre tracrRNA: TTATACCATACCAAGAACTATTATGGTTGCTATGATAAGGTCATAGGACCGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA 99
mario sequence: TGGTTGCTATGATAAGGTCATAGGAC 26


ora ha una lunghezza simile, e finisce in modo simile a quello di _S. pyogenes_  TTTTTTTaltre

In [65]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt3, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


GTCATAGTTCCCTAATAGCT-CTTGGTATGGTATAAT
            |||||| | |||||||||||||||
------------TAATAG-TTCTTGGTATGGTATAA-
  Score=22

rpt length: 36


In [87]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx((mario).reverse_complement(), rpt3)
print(pairwise2.format_alignment(*alignments[0]))

GTCCTATGA---CC-T--TA--TCAT----A-GC--A-ACCA-
|| | || |   || |  ||  || |    | |   | |  | 
GT-C-AT-AGTTCCCTAATAGCTC-TTGGTATG-GTATA--AT
  Score=19



In [105]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id LPEAHEHG_01071
----------------------------------------------------------------------

- contig name: NODE_190_length_65757_cov_25.3876
- Dataset:  ChengpingW_2017
- Bin:  ChengpingW_2017__AS43raw__bin.51
- SGB:  15299
----------------------------------------------------------------------

- Length:  99 
- position:  14652 14751
TTATACCATACCAAGAACTATTATGGTTGCTATGATAAGGTCATAGGACCGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA
